Copyright 2022 data cybernetics ssc GmbH.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import numpy as np
import pandas as pd
from baa_sdk import Client
from baa_sdk.models import JobConfig, Strategy

In [52]:
client = Client(api_key="kEDMTR04ep80JV0ql2OID9ASs57sIRi76Z83Pi0R", host="baa.jobs.api.dev.q-alchemy.com")
root = client.get_jobs_root()
root

In [53]:
pd.DataFrame.from_records(root.get_latest_job().get_result().get_result_evolution())

,TimeElapsedSeconds,FidelityLoss,CxGatesSaved,CxGates
0,0.000000,0.000000,0.0,0.0
1,2.001445,0.148767,27.0,0.0
2,5.069999,0.148767,27.0,0.0


In [54]:
import qiskit
qc = root.get_latest_job().get_result().get_best_node().to_circuit()
qiskit.transpile(qc, basis_gates=["rx", "ry", "rz", "cx", "id"]).draw(fold=-1)

global phase: 2.9815
      ┌────────────┐                    ┌─────────────┐  ┌───────┐       ┌──────────────┐                                   ┌─────────┐  ┌──────────────┐                    ┌────────────┐   ┌───────┐   
q_0: ─┤ Ry(1.5337) ├─────────────────■──┤ Rz(-1.1473) ├──┤ Ry(π) ├────■──┤ Rx(0.071337) ├───────────────────────────────■───┤ Rz(π/2) ├──┤ Ry(0.071337) ├─────────────────■──┤ Rz(2.5264) ├───┤ Ry(π) ├───
     ┌┴────────────┤  ┌─────────┐    │  └─────────────┘  └───────┘  ┌─┴─┐└┬────────────┬┘┌────────────┐ ┌────────────┐┌─┴─┐┌┴─────────┴─┐└┬────────────┬┘┌─────────────┐┌─┴─┐├────────────┤┌──┴───────┴──┐
q_1: ┤ Ry(-1.8678) ├──┤ Rz(π/2) ├────┼──────────────────────────────┤ X ├─┤ Rz(3.0716) ├─┤ Ry(1.7954) ├─┤ Rz(1.2659) ├┤ X ├┤ Rz(3.0522) ├─┤ Ry(1.8545) ├─┤ Rz(-1.8806) ├┤ X ├┤ Rx(2.8446) ├┤ Rz(0.53209) ├
     └─────────────┘  └─────────┘  ┌─┴─┐┌──────────────┐┌──────────┐└───┘ └┬──────────┬┘ ├───────────┬┘ └────────────┘└───┘└┬──────────┬┘┌┴────────────┤ └─────────────┘└───┘└────────────┘└─────────────┘
q_2: ──────────────────────────────┤ X ├┤ Rz(-0.36807) ├┤ Rx(-π/2) ├──■────┤ Rz(-π/2) ├──┤ Ry(2.862) ├──────────────────■───┤ Ry(-π/2) ├─┤ Rz(-1.0088) ├──────────────────────────────────────────────────
        ┌───────┐   ┌─────────────┐└───┘└──────────────┘└──────────┘  │    └──────────┘  └───────────┘                  │   └──────────┘ └─────────────┘                                                  
q_3: ───┤ Ry(π) ├───┤ Rz(-1.3589) ├───────────────────────────────────┼─────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────
      ┌─┴───────┴──┐└─────────────┘                                   │                                                 │                                                                                 
q_4: ─┤ Ry(1.4444) ├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────
     ┌┴────────────┤┌─────────────┐                                 ┌─┴─┐ ┌────────────┐ ┌────────────┐┌─────────────┐┌─┴─┐┌────────────┐┌─────────────┐                                                  
q_5: ┤ Rz(0.84865) ├┤ Rx(-2.8446) ├─────────────────────────────────┤ X ├─┤ Rz(1.4899) ├─┤ Ry(1.5599) ├┤ Rz(-1.8379) ├┤ X ├┤ Rx(1.2738) ├┤ Rz(-1.0426) ├──────────────────────────────────────────────────
     └─────────────┘└─────────────┘                                 └───┘ └────────────┘ └────────────┘└─────────────┘└───┘└────────────┘└─────────────┘                                                  
q_6: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Create and Configure a new Job

In [37]:
job = root.create_job()
job.rename("Carsten's Job")

config = job.get_config()
config.upload(JobConfig(
    max_fidelity_loss=0.15,
    strategy=Strategy.BRUTE_FORCE.value,
    use_low_rank=True
))
job_config = config.job_config()

state_vector = job.get_state_vector()
vector = np.load("../tests/data/test_baa_state.7.1.npy")
state_vector.upload_vector(vector)
downloaded_vector = state_vector.get_vector()

In [38]:
job.update().state, job.schedule()

('ReadyForProcessing', True)

In [44]:
print(job.update().state)

Completed


In [45]:
pd.DataFrame.from_records(job.get_result().get_result_evolution())

,TimeElapsedSeconds,FidelityLoss,CxGatesSaved,CxGates
0,0.000000,0.000000,0.0,0.0
1,2.001445,0.148767,27.0,0.0
2,5.069999,0.148767,27.0,0.0


In [49]:
nodes = job.get_result().get_result_nodes()
qc = nodes[0].to_circuit()
qiskit.transpile(qc, basis_gates=["rx", "ry", "rz", "cx", "id"]).draw(fold=-1)

global phase: 1.2845
     ┌─────────────┐                               ┌─────────────┐                                                                                              ┌──────────┐  ┌─────────┐                ┌───┐  ┌──────────┐     ┌───┐      ┌──────────┐ ┌─────────────┐    ┌───┐     ┌─────────────┐   ┌────────┐                                          ┌───┐     ┌───────────┐ ┌───────────┐     ┌───┐     ┌─────────────┐ ┌───────────┐┌─────────────┐    ┌───┐      ┌────────────┐┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
q_0: ┤ Ry(0.93385) ├──────────────────────■────────┤ Rz(-0.8781) ├───■──────────────────────────────────────────────────────────────────────────────────────■───┤ Rz(-π/2) ├──┤ Ry(π/2) ├────────────────┤ X ├──┤ Rx(-π/2) ├─────┤ X ├──────┤ Rz(-π/2) ├─┤ Ry(0.25402) ├────┤ X ├─────┤ Ry(-1.9502) ├───┤ Rz(-π) ├──────────────────────────────────────────┤ X ├─────┤ Ry(-2.75) ├─┤ Rx(1.602) ├─────┤ X ├─────┤ Rz(0.36182) ├─┤ Ry(1.591) ├┤ Rz(-1.5416) ├────┤ X ├──────┤ Rx(1.5396) ├┤ Rz(2.7181) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     └─┬──────────┬┘  ┌─────────┐         │        └─────────────┘   │                                    ┌───┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌┴──────────┴┐┌┴─────────┴─┐┌────────────┐└─┬─┘  └──────────┘     └─┬─┘      └──────────┘ └─────────────┘    └─┬─┘     └─┬──────────┬┘  ┌┴────────┤  ┌───┐ ┌──────────┐ ┌──────────────┐     └─┬─┘     └───────────┘ └───────────┘     └─┬─┘     └─────────────┘ └───────────┘└─────────────┘    └─┬─┘      └────────────┘└────────────┘                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
q_1: ──┤ Rz(-π/2) ├───┤ Ry(π/2) ├─────────┼──────────────────────────┼────────────────────────────────────┤ X ├┤ Rz(-2.1403) ├┤ Ry(1.8192) ├┤ Rz(2.0075) ├┤ X ├┤ Rz(2.5488) ├┤ Ry(1.0209) ├┤ Rz(3.0722) ├──■───────────────────────┼──────────

In [50]:
qc = nodes[1].to_circuit()
qiskit.transpile(qc, basis_gates=["rx", "ry", "rz", "cx", "id"]).draw(fold=-1)

global phase: 1.6767
     ┌─────────────┐               ┌─────────────┐┌────────┐                ┌──────────────┐┌──────────┐      ┌──────────┐┌────────────┐                     ┌─────────┐   ┌────────────┐
q_0: ┤ Ry(0.24334) ├───────■───────┤ Ry(-1.5017) ├┤ Rz(-π) ├───────■────────┤ Rz(-0.65371) ├┤ Ry(-π/2) ├──■───┤ Rz(-π/2) ├┤ Ry(3.0703) ├─────────────────■───┤ Ry(π/2) ├───┤ Rz(2.7311) ├
     └─────────────┘     ┌─┴─┐     ├─────────────┤└────────┘       │        ├─────────────┬┘├──────────┤┌─┴─┐┌┴──────────┤├────────────┤┌─────────────┐┌─┴─┐┌┴─────────┴─┐┌┴────────────┤
q_1: ────────────────────┤ X ├─────┤ Ry(0.26057) ├────■────────────┼────────┤ Ry(-1.3451) ├─┤ Rz(-π/2) ├┤ X ├┤ Rz(1.261) ├┤ Ry(1.2871) ├┤ Rz(-3.0522) ├┤ X ├┤ Rx(2.8446) ├┤ Rz(-1.5838) ├
                         └───┘     └─────────────┘    │          ┌─┴─┐     ┌┴─────────────┴┐├─────────┬┘└───┘└┬──────────┤├───────────┬┘└─────────────┘└───┘└┬──────────┬┘├─────────────┤
q_2: ─────────────────────────────────────────────────┼──────────┤ X ├─────┤ Rz(-0.013024) ├┤ Ry(π/2) ├───■───┤ Rz(-π/2) ├┤ Ry(2.862) ├──────────────────■───┤ Ry(-π/2) ├─┤ Rz(0.91709) ├
        ┌───────┐   ┌─────────────┐                   │          └───┘     └───────────────┘└─────────┘   │   └──────────┘└───────────┘                  │   └──────────┘ └─────────────┘
q_3: ───┤ Ry(π) ├───┤ Rz(-1.3589) ├───────────────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────┼───────────────────────────────
      ┌─┴───────┴──┐└─────────────┘                   │                                                   │                                              │                               
q_4: ─┤ Ry(1.4444) ├──────────────────────────────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────┼───────────────────────────────
      └────────────┘                                ┌─┴─┐   ┌─────────────┐   ┌─────────┐               ┌─┴─┐┌───────────┐┌────────────┐┌─────────────┐┌─┴─┐┌────────────┐┌─────────────┐
q_5: ───────────────────────────────────────────────┤ X ├───┤ Ry(-2.1475) ├───┤ Rz(π/2) ├───────────────┤ X ├┤ Rz(1.261) ├┤ Ry(1.2871) ├┤ Rz(-3.0522) ├┤ X ├┤ Rx(2.8446) ├┤ Rz(-2.9685) ├
                                                    └───┘   └─────────────┘   └─────────┘               └───┘└───────────┘└────────────┘└─────────────┘└───┘└────────────┘└─────────────┘
q_6: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────